In [ ]:
def start_signature():
    print("="*40)
    print("START OF NOTEBOOK — AT")
    print("="*40)

start_signature()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np

# lets look at the base table - transfers 
df_transfers = pd.read_csv(r'C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\transfers.csv')
df_transfers.head()

In [ ]:
# I want to know the unique number of patients and unique admissions 

print("the no of unique patients :", df_transfers["subject_id"].nunique())
print(" the no of unique admissions :", df_transfers["hadm_id"].nunique())
print(" the no of unique transfers :", df_transfers["transfer_id"].nunique())

In [ ]:
# i am gonna calculate our output variable now - time spent in ED. Now for that first I need only ED data , everything else not needed so need to remove that
# Secondly remove any blank values and then create a new variable called LOS_ED - length of stay in ED
# Goal is to predict how long a patient will stay in the ED during a single hospital visit, so we will  use the admission level.
# LOS_ED_houurs - defined as the time spent by a patient in emergency department calculated by the difference between intime and outtime 

units = ['Transplant']
df_transfers = df_transfers[df_transfers['careunit'].isin(units)].copy()
# df_transfers = df_transfers.dropna(subset =['intime','outtime'])

In [ ]:
df_transfers['intime'] = pd.to_datetime(df_transfers['intime'], errors='coerce')
df_transfers['outtime'] = pd.to_datetime(df_transfers['outtime'], errors='coerce')

df_transfers = df_transfers.dropna(subset=['intime','outtime'])
df_transfers = df_transfers[df_transfers['outtime'] >= df_transfers['intime']].copy()

In [ ]:
# now lets keep datetime format and calculate LOS_HC (hours)
df_transfers['LOS_Transplant'] = (df_transfers['outtime']-df_transfers['intime']).dt.total_seconds()/3600
df_transfers = df_transfers[df_transfers["LOS_Transplant"] > 0].copy()

In [ ]:
# lets aggregate at admission id level, need to sum the times spent in Emergency department and emergency department observation
# if a patient had multiple ED transfers hence why.
# keeping subject_id , so that if i want later can keep the count of patients 

df_transfers = (
    df_transfers.groupby("hadm_id", as_index=False)
    .agg({"LOS_Transplant": "sum", "subject_id": "first"})
    .rename(columns={"LOS_Transplant": "LOS_Transplant_hours"})
)

df_transfers["LOS_Transplant_hours"] = df_transfers["LOS_Transplant_hours"].round(2)

In [ ]:
print (df_transfers.head())
print("Transplant admissions:", df_transfers["hadm_id"].nunique())
print("Transplant patients:", df_transfers["subject_id"].nunique())

In [ ]:
df_transfers.info()
#df_transfers.isnull().any().any()
df_transfers.isnull().sum()


In [ ]:
df_transfers['LOS_Transplant_hours'].describe()

In [ ]:

plt.figure(figsize=(10,6))
sns.histplot(df_transfers['LOS_Transplant_hours'], bins=400, kde=True, color='blue')
plt.title('Distribution of 	Transplant Length of Stay (hours) with Density Curve')
plt.xlabel('LOS_Transplant_hours')
plt.ylabel('Density / Count')
plt.show()




In [ ]:
df_transfers['LOS_Transplant_hours'].describe(percentiles=[0.25, 0.5, 0.75, 0.90, 0.95, 0.99])


In [ ]:

plt.figure(figsize=(10,4))
sns.boxplot(x=df_transfers['LOS_Transplant_hours'], color='lightblue')
plt.title('Boxplot of Target Variable')
plt.xlabel('Target')
plt.show()


In [ ]:
Q1 = df_transfers['LOS_Transplant_hours'].quantile(0.25)
Q3 = df_transfers['LOS_Transplant_hours'].quantile(0.75)
IQR = Q3 - Q1

upper_limit = Q3 + 1.5 * IQR
lower_limit = Q1 - 1.5 * IQR

outliers = df_transfers[(df_transfers['LOS_Transplant_hours'] > upper_limit) | (df_transfers['LOS_Transplant_hours'] < lower_limit)]
print(f"Outlier count: {len(outliers)} ({len(outliers)/len(df_transfers)*100:.2f}% of total)")


In [ ]:
upper_limit = df_transfers['LOS_Transplant_hours'].quantile(0.75) + 1.5 * (df_transfers['LOS_Transplant_hours'].quantile(0.75) - df_transfers['LOS_Transplant_hours'].quantile(0.25))
print("Upper limit (IQR method):", upper_limit)

df_transfers.loc[df_transfers['LOS_Transplant_hours'] > upper_limit, 'LOS_Transplant_hours'].describe()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,6))

sns.histplot(df_transfers["LOS_Transplant_hours"], kde=True, bins=80, color="lightblue", stat="count", ax=axes[0])
axes[0].set_xlim(0, df_transfers["LOS_Transplant_hours"].quantile(0.99))
axes[0].set_title("Original Data (0–99th percentile)")

sns.histplot(df_transfers.loc[df_transfers["LOS_Transplant_hours"] <= upper_limit, "LOS_Transplant_hours"],
             kde=True, bins=80, color="salmon", stat="count", ax=axes[1])
axes[1].set_xlim(0, df_transfers["LOS_Transplant_hours"].quantile(0.99))
axes[1].set_title("Without IQR Outliers (0–99th percentile)")

plt.tight_layout()
plt.show()

In [ ]:
df_transfers['is_outlier'] = df_transfers['LOS_Transplant_hours'] > 24

plt.figure(figsize=(10,6))
sns.histplot(data=df_transfers, x='LOS_Transplant_hours', hue='is_outlier', bins=100, stat='density',
             palette={False: 'skyblue', True: 'salmon'}, kde=True)
plt.xlim(0, df_transfers['LOS_Transplant_hours'].quantile(0.99))
plt.title('Distribution — Highlighting Outliers (>24)')
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,6))

sns.histplot(df_transfers['LOS_Transplant_hours'], kde=True, bins=80, stat='count', color='skyblue', ax=axes[0])
axes[0].set_xlim(0, df_transfers['LOS_Transplant_hours'].quantile(0.99))
axes[0].set_title('Raw Target (0–99th percentile)')

sns.histplot(np.log1p(df_transfers['LOS_Transplant_hours']), kde=True, bins=80, stat='count', color='coral', ax=axes[1])
axes[1].set_title('Log(1+Target)')

plt.show()


In [ ]:
df_patients = pd.read_csv(r'C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\patients.csv')
df_patients.info()

In [ ]:
df_patients.head()

In [ ]:
df_patients.isnull().sum()

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df_patients['anchor_age'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Patient Age')
plt.xlabel('Age (years)')
plt.ylabel('Density / Count')
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='gender', data=df_patients, palette='pastel')
plt.title('Distribution of Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Patients')
plt.show()


In [ ]:
# lets keep only required columns in patients table
df_patients = df_patients.drop(columns=['anchor_year','anchor_year_group','dod'])

In [ ]:
df_merge1=df_transfers.merge(df_patients,on='subject_id',how='left')
df_merge1.head()

In [ ]:
df_merge1.info()

In [ ]:
df_merge1=df_merge1[df_merge1['anchor_age'] >=65].copy()


In [ ]:
print(df_merge1.shape)
print(df_merge1.head())

In [ ]:
plt.figure(figsize=(8,5))
sns.regplot(x='anchor_age', y='LOS_Transplant_hours', data=df_merge1, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
plt.title(' Transplant vs Age with Trend Line')
plt.xlabel('Age (years)')
plt.ylabel('LOS_Transplant_hours')
plt.show()


In [ ]:
corr = df_merge1['anchor_age'].corr(df_merge1['LOS_Transplant_hours'])
print(f"Correlation between age and Transplant LOS: {corr:.2f}")


In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='gender', y='LOS_Transplant_hours', data=df_merge1, palette='viridis')
plt.title('Transplant LOS by Gender')
plt.xlabel('Gender')
plt.ylabel('LOS_Transplant_hours')
plt.show()


In [ ]:
df_admissions = pd.read_csv(r'C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\admissions.csv')
df_admissions.head()

In [ ]:
df_admissions.info()

In [ ]:
# keep admittime and drop leakage columns
df_admissions["admittime"] = pd.to_datetime(df_admissions["admittime"], errors="coerce")

df_admissions = df_admissions.drop(columns=['dischtime','discharge_location','language',
                                            'edregtime','edouttime','hospital_expire_flag','deathtime'])

In [ ]:
df_admissions = df_admissions.dropna(subset=['marital_status'])


In [ ]:
df_admissions.isnull().sum()

In [ ]:
df_admissions.info()

In [ ]:
#lets understand about provider

df_admissions['admit_provider_id'].nunique()

In [ ]:
patients_per_doctor = df_admissions.groupby('admit_provider_id')['subject_id'].nunique().reset_index()
patients_per_doctor.rename(columns={'subject_id': 'unique_patients'}, inplace=True)

print(patients_per_doctor)

In [ ]:
df_merge2 = pd.merge(df_merge1, df_admissions, on=['hadm_id','subject_id'], how='inner')
print(df_merge2.shape)
#using inner here instead of left coz i need matching records from both tables.

In [ ]:
print(df_merge2.shape)

# admission time features
df_merge2["admit_hour"]  = df_merge2["admittime"].dt.hour
df_merge2["admit_day"]   = df_merge2["admittime"].dt.dayofweek
df_merge2["admit_month"] = df_merge2["admittime"].dt.month

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='admission_type', data=df_admissions, palette='Set2')
plt.title('Type of Admission - Distribution', pad=15)   
plt.xlabel('Admission_Type')
plt.ylabel('No of Patients')

# trying to rotate axis coz the titles were overlapping here
plt.xticks(rotation=15, ha='right') 
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='admission_location', data=df_admissions, palette='Set2')
plt.title(' Admission Location - Distribution', pad=20)   
plt.xlabel('Admission_Location')
plt.ylabel('No of Patients')

# trying to rotate axis coz the titles were overlapping here
plt.xticks(rotation=25, ha='right') 
plt.tight_layout()
plt.show()


In [ ]:
insurance_counts = df_admissions['insurance'].value_counts()

plt.figure(figsize=(7,7))
plt.pie(
    insurance_counts, 
    labels=insurance_counts.index, 
    autopct='%1.1f%%', 
    startangle=90, 
    colors=plt.cm.Pastel1.colors
)
plt.title('Insurance Type Distribution', pad=20)
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(y='insurance', x='LOS_Transplant_hours', data=df_merge2, palette='Set2')
plt.title('Transplant LOS by Insurance Type', pad=15)
plt.xlabel('LOS_Transplant_hours')
plt.ylabel('Insurance Type')
plt.xticks(rotation=30)  # Rotate labels if too long
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(y='admission_type', x='LOS_Transplant_hours', data=df_merge2, palette='Set2')
plt.title('Transplant LOS by Admission Type', pad=15)
plt.xlabel('LOS_Transplant_hours')
plt.ylabel('Admission Type')
plt.xticks(rotation=30)  # Rotate labels if too long
plt.show()


In [ ]:
# admit hour/day EDA
plt.figure(figsize=(10,5))
sns.boxplot(x="admit_hour", y="LOS_Transplant_hours", data=df_merge2)
plt.title("Transplant LOS by Hour of Admission", pad=15)
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x="admit_day", y="LOS_Transplant_hours", data=df_merge2)
plt.title("Transplant LOS by Day of Week", pad=15)
plt.show()

In [ ]:
file_path = r'C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\diagnoses_icd.csv.gz'
df_diagnoses = pd.read_csv(file_path, compression ='gzip')
df_diagnoses.head()


In [ ]:
print('There are {} unique ICD9 codes in this dataset.'.format(df_diagnoses['icd_code'].value_counts().count()))

In [ ]:
def categorize_icd9_short(code):
    """Return short ICD-9 category name"""
    if pd.isna(code):
        return 'misc'
    
    code_str = str(code).strip().upper()
    if code_str.startswith(('E', 'V')):
        return 'misc'  
    
    try:
        num = int(code_str[:3])
    except ValueError:
        return 'misc'
    
    ranges = [
        ((1, 139), 'infectious'),
        ((140, 239), 'neoplasms'),
        ((240, 279), 'endocrine'),
        ((280, 289), 'blood'),
        ((290, 319), 'mental'),
        ((320, 389), 'nervous'),
        ((390, 459), 'circulatory'),
        ((460, 519), 'respiratory'),
        ((520, 579), 'digestive'),
        ((580, 629), 'genitourinary'),
        ((630, 679), 'pregnancy'),
        ((680, 709), 'skin'),
        ((710, 739), 'muscular'),
        ((740, 759), 'congenital'),
        ((760, 779), 'prenatal'),
        ((780, 799), 'misc'),
        ((800, 999), 'injury')
    ]
    
    for (low, high), label in ranges:
        if low <= num <= high:
            return label
    return 'misc'

In [ ]:
df_diagnoses['icd_category'] = df_diagnoses['icd_code'].apply(categorize_icd9_short)

diag_counts = (
    df_diagnoses.groupby(['hadm_id', 'icd_category'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

In [ ]:
df_merge2 = df_merge2.merge(diag_counts, on="hadm_id", how="left").fillna(0)


In [ ]:

df_merge2.fillna(0, inplace=True)

for col in diag_counts.columns:
    if col != 'hadm_id':
        df_merge2[col] = df_merge2[col].astype(int)


print(df_merge2.shape)
df_merge2.info()

In [ ]:
df_diag = df_diagnoses[["hadm_id", "icd_code", "icd_version"]].copy()

def normalize_icd(code):
    if pd.isna(code):
        return ""
    return str(code).strip().upper().replace(".", "")

df_diag["icd_code_clean"] = df_diag["icd_code"].apply(normalize_icd)
df_diag["is_icd10"] = (df_diag["icd_version"] == 10)

In [ ]:
def icd3(row):
    c = row["icd_code_clean"]
    if c == "":
        return ""
    if row["is_icd10"]:
        return c[:3]  # ICD-10 prefix
    digits = "".join(ch for ch in c if ch.isdigit())
    return digits[:3] if len(digits) >= 3 else digits

df_diag["icd3"] = df_diag.apply(icd3, axis=1)

# better complexity features
diag_complex = (
    df_diag.groupby("hadm_id")
    .agg(
        n_icd_codes=("icd_code_clean", "nunique"),
        n_unique_icd3=("icd3", lambda x: x.nunique())
    )
    .reset_index()
)

In [ ]:
# Transplant-focused ICD flags
ICD10_PREFIX_TX = {
    "has_transplant_status": {"Z94"},
    "has_transplant_complication": {"T86"},
    "has_ckd": {"N18"},
    "has_aki": {"N17"},
    "has_liver_disease": {"K70","K71","K72","K73","K74","K76"},
    "has_diabetes": {"E10","E11"},
    "has_htn": {"I10","I11","I12","I13"},
    "has_sepsis": {"A41"},
}

ICD9_RANGES_TX = {
    "has_transplant_complication": [(996, 996)],  # 996.xx
    "has_ckd": [(585, 585)],
    "has_aki": [(584, 584)],
    "has_liver_disease": [(570, 573)],
    "has_diabetes": [(250, 250)],
    "has_htn": [(401, 405)],
    "has_sepsis": [(995, 995)],  # rough
}

In [ ]:
def flag_icd10(icd3_list, prefixes):
    return int(any(str(v) in prefixes for v in icd3_list if str(v)))

def flag_icd9(icd3_list, ranges):
    nums = [int(v) for v in icd3_list if str(v).isdigit()]
    for lo, hi in ranges:
        if any(lo <= n <= hi for n in nums):
            return 1
    return 0

flags_rows = []
for hadm_id, g in df_diag.groupby("hadm_id"):
    icd3_list = g["icd3"].dropna().tolist()
    out = {"hadm_id": hadm_id}

    for feat, prefset in ICD10_PREFIX_TX.items():
        out[feat] = flag_icd10(icd3_list, prefset)

    for feat, ranges in ICD9_RANGES_TX.items():
        out[feat] = max(out.get(feat, 0), flag_icd9(icd3_list, ranges))

    flags_rows.append(out)

diag_flags = pd.DataFrame(flags_rows)

In [ ]:
df_merge2 = df_merge2.merge(diag_complex, on="hadm_id", how="left")
df_merge2 = df_merge2.merge(diag_flags, on="hadm_id", how="left")

new_cols = ["n_icd_codes", "n_unique_icd3"] + list(ICD10_PREFIX_TX.keys())
for c in new_cols:
    if c in df_merge2.columns:
        df_merge2[c] = df_merge2[c].fillna(0).astype(float)

# extra combined feature
df_merge2["any_transplant_related_dx"] = (
    (df_merge2["has_transplant_status"] > 0) |
    (df_merge2["has_transplant_complication"] > 0)
).astype(int)

In [ ]:
print("✅ Added transplant dx features:", new_cols + ["any_transplant_related_dx"])
print("df_merge2 shape:", df_merge2.shape)

In [ ]:
tx_hadm_ids = set(df_merge2["hadm_id"].dropna().unique())
print("Transplant admissions:", len(tx_hadm_ids))

In [ ]:
DLABITEMS_PATH = r"C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\d_labitems.csv"
df_labitems = pd.read_csv(DLABITEMS_PATH, usecols=["itemid", "label"])

TRANSPLANT_LABS = {
    "Creatinine", "Urea Nitrogen", "Glucose",
    "Sodium", "Potassium", "Chloride", "Bicarbonate",
    "White Blood Cells", "Hemoglobin", "Platelet Count",
    "Bilirubin, Total", "AST (SGOT)", "ALT (SGPT)", "Alkaline Phosphatase",
    "INR(PT)", "PT", "PTT"
}

tx_labitems = df_labitems[df_labitems["label"].isin(TRANSPLANT_LABS)].copy()
tx_lab_itemids = set(tx_labitems["itemid"].unique())

print("Selected transplant lab labels found:")
print(tx_labitems["label"].value_counts())
print("ItemIDs to keep:", tx_lab_itemids)

In [ ]:
adm_time_map = (
    df_admissions[["hadm_id", "admittime"]]
    .assign(admittime=lambda x: pd.to_datetime(x["admittime"], errors="coerce"))
    .set_index("hadm_id")["admittime"]
    .to_dict()
)

In [ ]:
import time
from collections import defaultdict

LABEVENTS_PATH = r"C:\Users\arpitha_work\Downloads\TRU MSCDS\Sem 3\Graduate Project\MIMIC\mimic-iv-3.1\hosp\labevents.csv.gz"

EARLY_WINDOW_HOURS = 12
CHUNKSIZE = 50_000

t0 = time.time()
chunk_counter = 0
rows_kept_total = 0

sum_dict   = defaultdict(float)
count_dict = defaultdict(int)
min_dict   = defaultdict(lambda: np.inf)
max_dict   = defaultdict(lambda: -np.inf)

for chunk in pd.read_csv(
    LABEVENTS_PATH,
    compression="gzip",
    usecols=["hadm_id", "itemid", "charttime", "valuenum"],
    chunksize=CHUNKSIZE
):
    chunk_counter += 1

    chunk = chunk[
        chunk["hadm_id"].isin(tx_hadm_ids) &
        chunk["itemid"].isin(tx_lab_itemids)
    ].dropna(subset=["charttime", "valuenum"])

    if chunk.empty:
        if chunk_counter % 50 == 0:
            print(f"chunks={chunk_counter}, kept_rows={rows_kept_total}, elapsed={(time.time()-t0)/60:.1f} min")
        continue

    chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")
    chunk = chunk.dropna(subset=["charttime"])

    chunk["admittime"] = chunk["hadm_id"].map(adm_time_map)
    chunk = chunk.dropna(subset=["admittime"])

    hours_from_admit = (chunk["charttime"] - chunk["admittime"]).dt.total_seconds() / 3600.0
    chunk = chunk[(hours_from_admit >= 0) & (hours_from_admit <= EARLY_WINDOW_HOURS)]

    if chunk.empty:
        if chunk_counter % 50 == 0:
            print(f"chunks={chunk_counter}, kept_rows={rows_kept_total}, elapsed={(time.time()-t0)/60:.1f} min")
        continue

    rows_kept_total += len(chunk)

    g = chunk.groupby(["hadm_id", "itemid"])["valuenum"]
    s = g.sum()
    c = g.count()
    mn = g.min()
    mx = g.max()

    for k, v in s.items():
        sum_dict[k] += float(v)
    for k, v in c.items():
        count_dict[k] += int(v)
    for k, v in mn.items():
        min_dict[k] = min(min_dict[k], float(v))
    for k, v in mx.items():
        max_dict[k] = max(max_dict[k], float(v))

    if chunk_counter % 50 == 0:
        print(f"chunks={chunk_counter}, kept_rows={rows_kept_total}, elapsed={(time.time()-t0)/60:.1f} min")

print("Finished streaming labevents")
print("Total chunks:", chunk_counter)
print("Rows kept:", rows_kept_total)
print("Total minutes:", (time.time() - t0) / 60)


In [ ]:
rows = []
for (hadm_id, itemid), cnt in count_dict.items():
    rows.append({
        "hadm_id": hadm_id,
        "itemid": itemid,
        "lab_mean": sum_dict[(hadm_id, itemid)] / cnt,
        "lab_min":  min_dict[(hadm_id, itemid)],
        "lab_max":  max_dict[(hadm_id, itemid)],
        "lab_count": cnt
    })

lab_agg = pd.DataFrame(rows)
lab_agg = lab_agg.merge(df_labitems, on="itemid", how="left")

print("\nLab long table:", lab_agg.shape)
print(lab_agg.head())

lab_agg.to_csv("tx_lab_agg_long.csv", index=False)
print("Saved: tx_lab_agg_long.csv")

lab_agg["lab_sum"] = lab_agg["lab_mean"] * lab_agg["lab_count"]

lab_collapsed = (
    lab_agg
    .groupby(["hadm_id", "label"], as_index=False)
    .agg(
        lab_sum=("lab_sum", "sum"),
        lab_count=("lab_count", "sum"),
        lab_min=("lab_min", "min"),
        lab_max=("lab_max", "max")
    )
)
lab_collapsed["lab_mean"] = lab_collapsed["lab_sum"] / lab_collapsed["lab_count"]

lab_wide = lab_collapsed.pivot_table(
    index="hadm_id",
    columns="label",
    values=["lab_mean","lab_min","lab_max","lab_count"],
    aggfunc="first"
)
lab_wide.columns = [f"{stat}_{label}".replace(" ", "_") for stat, label in lab_wide.columns]
lab_wide = lab_wide.reset_index()

print("\nLab wide features:", lab_wide.shape)
print(lab_wide.head())

df_merge2 = df_merge2.merge(lab_wide, on="hadm_id", how="left")
print("\n✅ df_merge2 shape after merging labs:", df_merge2.shape)


In [ ]:


# ------------------------------
 #DERIVED TRANSPLANT LAB FEATURES 
# ------------------------------
import numpy as np

def safe_div(a, b):
    return np.where((b == 0) | pd.isna(b), np.nan, a / b)

# Liver injury signal
if "lab_mean_AST_(SGOT)" in df_merge2.columns and "lab_mean_ALT_(SGPT)" in df_merge2.columns:
    df_merge2["ast_alt_ratio"] = safe_div(df_merge2["lab_mean_AST_(SGOT)"], df_merge2["lab_mean_ALT_(SGPT)"])

# Renal dysfunction signal
if "lab_mean_Creatinine" in df_merge2.columns and "lab_mean_Urea_Nitrogen" in df_merge2.columns:
    df_merge2["bun_creatinine_ratio"] = safe_div(df_merge2["lab_mean_Urea_Nitrogen"], df_merge2["lab_mean_Creatinine"])

# Coagulation severity proxy
if "lab_mean_INR(PT)" in df_merge2.columns:
    df_merge2["inr_high_flag"] = (df_merge2["lab_mean_INR(PT)"] >= 1.5).astype(int)

print("✅ Added derived lab features (if available):",
      [c for c in ["ast_alt_ratio","bun_creatinine_ratio","inr_high_flag"] if c in df_merge2.columns])


In [ ]:
adm = df_admissions[["hadm_id","subject_id","admittime"]].copy()
adm["admittime"] = pd.to_datetime(adm["admittime"], errors="coerce")
adm = adm.sort_values(["subject_id","admittime"]).copy()

adm["prev_adm_count"] = adm.groupby("subject_id").cumcount()
adm["prev_admittime"] = adm.groupby("subject_id")["admittime"].shift(1)

adm["days_since_last_adm"] = (adm["admittime"] - adm["prev_admittime"]).dt.total_seconds() / (3600*24)


In [ ]:
hist_feats = adm[["hadm_id","prev_adm_count","days_since_last_adm"]].copy()

df_merge2 = df_merge2.merge(hist_feats, on="hadm_id", how="left")
df_merge2["prev_adm_count"] = df_merge2["prev_adm_count"].fillna(0).astype(float)
df_merge2["days_since_last_adm"] = df_merge2["days_since_last_adm"].fillna(-1).astype(float)


In [ ]:

print(df_merge2[["prev_adm_count","days_since_last_adm"]].describe())
print("df_merge2 shape:", df_merge2.shape)

In [ ]:
TARGET_COL = "LOS_Transplant_hours"

df_merge2 = df_merge2[df_merge2[TARGET_COL].notna()].copy()
df_merge2 = df_merge2[df_merge2[TARGET_COL] > 0].copy()

# Quantile-based bins (3 classes) — safer for transplant
q1, q2 = df_merge2[TARGET_COL].quantile([0.33, 0.66])
bins = [0, q1, q2, np.inf]
labels = [0, 1, 2]  # 0=short, 1=medium, 2=long

df_merge2["los_class"] = pd.cut(df_merge2[TARGET_COL], bins=bins, labels=labels, right=False)
df_merge2 = df_merge2[df_merge2["los_class"].notna()].copy()
df_merge2["los_class"] = df_merge2["los_class"].astype(int)

print("Bins used:", bins)
print(df_merge2["los_class"].value_counts(normalize=True).sort_index())


In [ ]:
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, balanced_accuracy_score

X = df_merge2.drop(columns=[TARGET_COL, "los_class", "hadm_id", "subject_id"], errors="ignore").copy()
y = df_merge2["los_class"].astype(int).copy()

In [ ]:
# datetime -> engineered time parts
datetime_cols = X.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns.tolist()
for col in datetime_cols:
    dt = pd.to_datetime(X[col], errors="coerce")
    X[col + "_hour"] = dt.dt.hour
    X[col + "_weekday"] = dt.dt.weekday
    X[col + "_month"] = dt.dt.month
X.drop(columns=datetime_cols, inplace=True, errors="ignore")

# categorical cleanup
cat_like = X.select_dtypes(include=["object", "string", "category"]).columns
X[cat_like] = X[cat_like].astype(str)
X[cat_like] = X[cat_like].replace({"nan": np.nan, "NaT": np.nan, "None": np.nan})

In [ ]:
# ✅ patient-level split (prevents leakage)
groups = df_merge2["subject_id"]

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()

num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop",
    sparse_threshold=0.0
)

In [ ]:
# Models
rf = RandomForestClassifier(
    n_estimators=200, max_depth=12, min_samples_leaf=5,
    random_state=42, n_jobs=1, class_weight="balanced"
)

et = ExtraTreesClassifier(
    n_estimators=300, max_depth=12, min_samples_leaf=5,
    random_state=42, n_jobs=1, class_weight="balanced"
)

gbr = GradientBoostingClassifier(
    n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
)

logreg = LogisticRegression(max_iter=2000, class_weight="balanced")

voting_soft = VotingClassifier(
    estimators=[("lr", logreg), ("rf", rf), ("et", et), ("gbr", gbr)],
    voting="soft"
)
models = {
    "LogisticRegression": logreg,
    "RandomForest": rf,
    "ExtraTrees": et,
    "GradientBoostingClassifier": gbr,
    "VotingSoft": voting_soft
}

In [ ]:
results = {}

for name, model in models.items():
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])

    print(f"\nTraining: {name}")
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")
    f1_weighted = f1_score(y_test, y_pred, average="weighted")

    results[name] = {
        "Accuracy": acc,
        "Balanced_Acc": bal_acc,
        "F1_macro": f1_macro,
        "F1_weighted": f1_weighted
    }

    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

results_df = pd.DataFrame(results).T.sort_values("F1_macro", ascending=False)
print("\n===== RESULTS (sorted by F1_macro) =====")
print(results_df)

In [ ]:
def end_signature():
    print("="*40)
    print("END OF NOTEBOOK — AT")
    print("="*40)

end_signature()
